## Paper LLM - Cobertura arbórea y captura de dióxido de carbono en los parques urbanos


# Principales extenciones necesarias:

In [ ]:
!pip install openai tiktoken pypdf chromadb langchain

In [ ]:
!pip install PyMuPDF --quiet

In [ ]:
!pip install pypdf


In [ ]:
!pip install --upgrade openai

In [ ]:
import fitz
import tiktoken

In [ ]:
pdf_path = "IIGeo_Arevalo.pdf"

In [ ]:
# Paso 1: Extraer texto desde PDF

def extraer_texto_pdf(ruta_pdf):
    doc = fitz.open(ruta_pdf)
    texto = ""
    for pagina in doc:
        texto += pagina.get_text()
    doc.close()
    return texto

tokenizer = tiktoken.get_encoding("cl100k_base")   
def tokenCounter(text):
    return len(tokenizer.encode(text))
bookText = extraer_texto_pdf(pdf_path)

In [ ]:
print(bookText)

© Los autores. Este artículo es publicado por la Revista del Instituto de investigación de la Facultad de minas, metalurgia y ciencias 
geográficas de la Universidad Nacional Mayor de San Marcos. Este es un artículo de acceso abierto, distribuido bajo los términos de la 
licencia Creative Commons Atribución 4.0 Internacional (CC BY 4.0) [https://creativecommons.org/licenses/by/4.0/deed.es] que permite 
el uso, distribución y reproducción en cualquier medio, siempre que la obra original sea debidamente citada de su fuente original. Para 
mas información, por favor póngase en contacto con iigeo@unmsm.edu.pe
Rev. Inst. investig. Fac. minas metal. cienc. geogr. vol 24 n° 47, 2021: 13 - 18	
ISSN-L:1561-0888
Cobertura arbórea y captura de dióxido de carbono en los 
parques urbanos. Caso: Lima Norte
Tree cover and carbone dioxide capture in urban parks case: Northern Lima
Walter Aparicio Arévalo Gómez 1 , Francisco Alejandro Alcántara Boza 2
Recibido: 14/10/2020 - Aprobado: 26/04/2021 – Publi

### 3. División del texto en fragmentos (chunks)
Se divide el texto extraído en fragmentos manejables usando `RecursiveCharacterTextSplitter` de LangChain, considerando el número de tokens:
```python

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

textSplitter = RecursiveCharacterTextSplitter(
    chunk_size=8192,
    chunk_overlap=50,
    length_function=tokenCounter,
    separators = ["\n\n", ".", "\n", " "]
)

chunks = textSplitter.create_documents(
    [bookText],
        metadatas=[{
        "title": "Cobertura arbórea y captura de dióxido de carbono en los parques urbanos. Caso: Lima Norte",
        "title_en": "Tree cover and carbon dioxide capture in urban parks. Case: Northern Lima",
        "authors": "Walter Aparicio Arévalo Gómez, Francisco Alejandro Alcántara Boza",
        "journal": "Rev. Inst. investig. Fac. minas metal. cienc. geogr.",
        "volume": "24",
        "number": "47",
        "year": "2021"
    }]
)

len(chunks)

1

In [ ]:
Texto_arboleda = chunks[0].page_content


In [ ]:
print(chunks[0].page_content)

© Los autores. Este artículo es publicado por la Revista del Instituto de investigación de la Facultad de minas, metalurgia y ciencias 
geográficas de la Universidad Nacional Mayor de San Marcos. Este es un artículo de acceso abierto, distribuido bajo los términos de la 
licencia Creative Commons Atribución 4.0 Internacional (CC BY 4.0) [https://creativecommons.org/licenses/by/4.0/deed.es] que permite 
el uso, distribución y reproducción en cualquier medio, siempre que la obra original sea debidamente citada de su fuente original. Para 
mas información, por favor póngase en contacto con iigeo@unmsm.edu.pe
Rev . Inst . InvestIg . Fac. mInas  metal . cIenc . geogR . vol  24 n ° 47, 2021: 13 - 18  Issn-l:1561-0888
Cobertura arbórea y captura de dióxido de carbono en los 
parques urbanos. Caso: Lima Norte
Tree cover and carbone dioxide capture in urban parks case: Northern Lima
Walter Aparicio Arévalo Gómez 1 , Francisco Alejandro Alcántara Boza 2
Recibido: 14/10/2020 - Aprobado: 26/04/202

In [ ]:
from openai import OpenAI
from getpass import getpass

# Pedir clave y limpiar espacios
api_key = getpass("Ingresa tu API key de OpenAI: ").strip()

# Crear el cliente
client = OpenAI(api_key=api_key)

# Probar la conexión
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "¿Estás funcionando?"}]
)

print(response.choices[0].message.content)

Ingresa tu API key de OpenAI: ··········
Sí, estoy funcionando correctamente. ¿En qué puedo ayudarte hoy?


In [ ]:
def ask_gpt(prompt, temperature=0.2):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        temperature=temperature,
        messages=[
            {"role": "system", "content": "Eres un experto en análisis ambiental y políticas públicas."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

### 6. Prompts para análisis y recomendaciones
Se construyen prompts detallados para que el modelo clasifique la eficiencia de los parques y recomiende especies óptimas para reforestación, usando los datos extraídos del PDF.


In [ ]:
prompt = f"""
A partir de los datos de porcentaje de cobertura arbórea y kilogramos de CO₂ capturado por metro cuadrado por año (kg CO₂/m²·año) en los 32 parques analizados (Tabla 1), clasifica los parques urbanos de Lima Norte en categorías de eficiencia. Define tres niveles (alta, media y baja eficiencia), indicando los rangos de cobertura arbórea y kg CO₂/m²·año que corresponden a cada categoría. Además, explica brevemente los factores que influyen en que un parque pase de una categoría a otra (por ejemplo, relación cobertura–captura, tamaño del parque, composición de especies, estado de conservación).

Instrucciones:

Recopilación de datos:
a. Extrae de la Tabla 1 los valores de “% Cobertura Arbórea” y “kg CO₂/m²·año” para cada parque (códigos CO1, CO2, …, SM9).
b. Asocia a cada parque sus dos indicadores principales: porcentaje de cobertura arbórea y captura anual de CO₂.

Definición de niveles de eficiencia:
a. Basándote en los valores mínimo, máximo y el promedio general de captura (0.34 kg CO₂/m²·año), propone tres categorías:
– Alta eficiencia: cobertura ≥ 50 % y captura ≥ 0.55 kg CO₂/m²·año.
– Eficiencia media: cobertura entre 25 %–49 % y captura entre 0.25 – 0.54 kg CO₂/m²·año.
– Baja eficiencia: cobertura < 25 % y captura < 0.25 kg CO₂/m²·año.
b. Justifica cada umbral señalando cómo se relacionan con los valores extremos de la tabla y el promedio general de captura.

Asignación de parques a categorías:
a. Enumera los códigos de parque que correspondan a cada nivel de eficiencia (alta, media, baja).
b. Para cada categoría, agrega un breve comentario que incluya:
– Número de parques en esa categoría.
– Rango de áreas (en m²) de los parques asignados.
– Rango de porcentajes de cobertura arbórea observados.

Análisis de factores determinantes:
Selecciona y explica al menos tres variables que expliquen por qué un parque puede pasar de baja a alta eficiencia:
a. Relación cobertura–captura: por ejemplo, menciona que parques como IN1 e IN4 presentan ≥ 0.67 kg CO₂/m²·año debido a cobertura ≥ 62 %.
b. Tamaño del parque: compara los efectos en captura de CO₂ entre parques pequeños (≤ 2 500 m²) y parques grandes (> 10 000 m²).
c. Estado de conservación y composición de especies: describe cómo la presencia de árboles maduros versus plantones puede influir en el coeficiente de captura.

Formato de presentación:
a. Incluye una tabla o lista con las tres categorías y los parques asignados a cada una.
b. (Opcional) Agrega un gráfico de barras o un diagrama de dispersión donde el eje X sea porcentaje de cobertura arbórea, el eje Y sea captura de CO₂ (kg CO₂/m²·año) y se muestren líneas o áreas que distingan los umbrales entre baja, media y alta eficiencia.
c. Escribe un breve texto (2–3 párrafos) que explique cómo utilizar esta clasificación para priorizar intervenciones en los parques: por ejemplo, cuáles requieren reforestación, aumento de cobertura o cambio de especies, y cuáles necesitan mantenimiento (riego, control de plagas) para mejorar su eficiencia.




Texto:
{bookText}
"""
respuesta_eficiencia = ask_gpt(prompt)

In [ ]:
prompt = f"""
A partir de las recomendaciones del estudio y la literatura citada (Brescia De Fort, 2010; Domínguez, 2016), elabora una clasificación comparativa de especies arbóreas sugeridas para parques urbanos de Lima Norte. Para cada especie propuesta (Schinus molle, Schinus terebinthifolius, Grevillea robusta, Eucalyptus globulus, Ficus benjamina), indica:

Captura anual promedio de CO₂ (kg CO₂/año) según la literatura.

Adaptabilidad a condiciones áridas y escasez de agua en Lima Norte.

Cobertura esperada de copa (% de área verde) y requerimientos de mantenimiento.

Finalmente, recomienda las tres especies “óptimas” que combinan mejor alta captura de CO₂, bajo requerimiento hídrico y viabilidad de implementación en espacios urbanos.

Instrucciones bien estructuradas:

Listado de especies y datos de captura:
a. Extrae de la sección VI (Recomendación) los valores de captura anual promedio:
– Schinus molle: 182.6 kg CO₂/año.
– Schinus terebinthifolius: 74.4 kg CO₂/año.
– Grevillea robusta: 229.0 kg CO₂/año.
– Eucalyptus globulus: 162.3 kg CO₂/año.
– Ficus benjamina: 63.9 kg CO₂/año.

Adaptabilidad climática y uso de agua:
a. Para cada especie, resume su tolerancia a la sequía con base en estudios citados o literatura local.
b. Clasifica la “necesidad de riego” en tres niveles:
– Baja: puede sobrevivir con precipitaciones mínimas.
– Media: requiere riego moderado (< 10 L/árbol/semana).
– Alta: necesita riego frecuente (> 10 L/árbol/semana).
c. Menciona aspectos de resistencia a suelos compactados y polución urbana.

Cobertura de copa y mantenimiento:
a. Estima, a partir de literatura adicional (por ejemplo, manuales de arboricultura urbana), el diámetro de copa maduro (en metros) para cada especie.
b. Traduce el diámetro a porcentaje de cobertura de parque si se conocen densidades típicas (por ejemplo, “3 árboles de Grevillea robusta pueden cubrir aproximadamente 20 % de un parque de 2 500 m²”).
c. Incluye una columna con frecuencia aproximada de poda o control sanitario (cada 6 meses, anual, etc.).

Comparación y puntuación:
a. Crea una tabla donde cada fila sea una especie y las columnas sean:
– Captura anual (kg CO₂/año).
– Nivel de adaptabilidad a sequía (Baja/Media/Alta).
– Cobertura aproximada de copa (%) para un parque modelo (por ejemplo, 2 500 m²).
– Mantenimiento (Bajo/Medio/Alto costo).
b. Asigna una puntuación de 1 a 5 a cada criterio (donde 5 es ideal/óptimo).
c. Calcula una puntuación final (suma o promedio ponderado) para cada especie.

Recomendaciones finales:
a. Selecciona las tres especies con mayor puntaje.
b. Para cada especie recomendada, incluye:
– Pros (por ejemplo, “alta captura, tolerante a sequía, copa densa”).
– Contras (por ejemplo, “raíces invasivas”, “podas frecuentes”).
– Estrategia de implementación: sugiere densidad de plantación (árboles/ha) y periodicidad de riego inicial (primer año).

Formato de presentación:
a. Presenta una tabla comparativa clara con todas las columnas indicadas.
b. Escribe un texto explicativo de 2–3 párrafos sobre por qué esas tres especies resultan óptimas para parques urbanos de Lima Norte.
c. Incluye una breve sección titulada “Pasos recomendados” para un programa de reforestación urbana en Lima Norte, detallando acciones concretas y cronograma básico.


Texto:
{bookText}
"""
respuesta_especies_optimas = ask_gpt(prompt)

### 7. Presentación de resultados
Se imprimen y analizan las respuestas generadas por el modelo, que incluyen tablas, clasificaciones y recomendaciones para políticas públicas.


In [49]:
print(respuesta_eficiencia)

### Clasificación de parques urbanos de Lima Norte en categorías de eficiencia:

#### Definición de niveles de eficiencia:
- **Alta eficiencia:** cobertura ≥ 50 % y captura ≥ 0.55 kg CO₂/m²·año.
- **Eficiencia media:** cobertura entre 25 %–49 % y captura entre 0.25 – 0.54 kg CO₂/m²·año.
- **Baja eficiencia:** cobertura < 25 % y captura < 0.25 kg CO₂/m²·año.

#### Asignación de parques a categorías:
- **Alta eficiencia:** IN1, IN4, LO1, SM1, SM8.
- **Eficiencia media:** CO1, CO4, CO9, IN2, IN5, LO3, LO5, LO7, LO9, SM3, SM5.
- **Baja eficiencia:** CO2, CO3, CO6, CO7, CO8, IN3, LO2, LO4, LO6, SM2, SM4, SM6, SM7, SM9.

#### Análisis de factores determinantes:
1. **Relación cobertura–captura:** Parques como IN1 e IN4 presentan altos valores de captura debido a su alta cobertura arbórea.
2. **Tamaño del parque:** Parques más grandes tienden a tener mayor captura de CO₂ debido a una mayor cantidad de árboles.
3. **Estado de conservación y composición de especies:** La presencia de árboles mad

In [ ]:
prompt_politicas = f"""
Usa la siguiente información extraída:

1. Diagnósticos y eficiencia:
{respuesta_eficiencia}

2. Información sobre especies optimas:
{respuesta_especies_optimas}



Con base en lo anterior, responde:
¿Cómo puede esta información apoyar o influir en la formulación de políticas públicas para mejorar la captura de carbono en áreas urbanas? Por favor, da recomendaciones específicas y justificadas.
"""

respuesta_politicas = ask_gpt(prompt_politicas)

In [ ]:
print(respuesta_politicas)

La información proporcionada sobre la clasificación de parques urbanos en Lima Norte según su eficiencia en la captura de carbono, así como la comparativa de especies arbóreas óptimas para la reforestación, puede ser de gran utilidad para la formulación de políticas públicas orientadas a mejorar la captura de carbono en áreas urbanas. A continuación, se presentan algunas recomendaciones específicas y justificadas:

1. **Priorización de intervenciones en parques según su eficiencia:** Las autoridades locales pueden utilizar la clasificación de parques en categorías de eficiencia para priorizar las intervenciones de reforestación. Se recomienda enfocar los recursos en parques con baja eficiencia, donde se necesita aumentar la cobertura arbórea con especies adecuadas. Esto contribuirá a mejorar la captura de carbono en áreas urbanas y a maximizar el impacto ambiental de las acciones de reforestación.

2. **Promoción de especies óptimas para la reforestación:** Las especies arbóreas óptima

### Debate sobre decisiones de LLM

En las últimas décadas, los modelos de lenguaje a gran escala (LLM) han pasado de ser curiosidades académicas a herramientas influyentes en aplicaciones reales: desde asistentes virtuales hasta apoyo en análisis de datos y redacción automatizada. Su capacidad para procesar grandes volúmenes de texto y generar respuestas coherentes ha levantado expectativas sobre su papel en la toma de decisiones en ámbitos tan diversos como la medicina, los negocios y la gobernanza. Sin embargo, esta adopción conlleva interrogantes filosóficos y prácticos: ¿hasta qué punto podemos confiar en un sistema que no “comprende” en sentido humano? ¿Cómo equilibrar su eficiencia con la necesidad de juicio crítico?

---

### 2. **Validez**
**2.1. Casos de uso donde los LLM pueden considerarse fiables**  
- **Velocidad de procesamiento**  
  - Procesan y sintetizan miles de documentos en segundos, un esfuerzo que para un ser humano tomaría horas o días.  
  - En tareas como resúmenes rápidos de literatura o generación de borradores, facilitan la labor preliminar de investigación.  
- **Capacidad de síntesis de información**  
  - Identifican patrones estadísticos en textos masivos, ofreciendo resúmenes de tendencias o consensos frecuentes en el corpus.  
  - Pueden generar comparativas de argumentos encontrados en múltiples fuentes, útil para orientar discusiones o propuestas de políticas.  
- **Criterios epistemológicos**  
  - **Epistemología pragmática**: si la meta es acceder a información rápida y razonablemente coherente, los LLM satisfacen un criterio pragmático de “conocimiento útil”.  
  - **Validez estadística**: su fiabilidad depende de la calidad de los datos de entrenamiento y de la robustez de las actualizaciones.  

**2.2. Argumentos a favor**  
- **Reducción de sesgo humano inicial**: al procesar datos masivos, pueden atenuar (aunque no eliminar) ciertos prejuicios individuales, ofreciendo una visión más amplia de fuentes diversas.  
- **Consistencia en respuestas**: en ámbitos donde se requiera uniformidad (por ejemplo, protocolos médicos o guías de estilo), los LLM mantienen un estilo y criterio estandarizado.  
- **Accesibilidad democratizada**: democratizan el acceso a conocimiento especializado, permitiendo que usuarios sin formación profunda obtengan resúmenes y referencias relevantes.

**2.3. Criterios para considerar un LLM adecuado**  
- **Transparencia en su entrenamiento**: conocer el origen de los datos reduce incertidumbres sobre sesgos de fondo.  
- **Actualización constante**: en dominios que evolucionan rápidamente (salud, finanzas), un modelo obsoleto pierde validez.  
- **Evaluación empírica continua**: métricas de calidad (exactitud, coherencia, ausencia de contradicciones) deben revisarse periódicamente.

---

### 3. **Limitaciones**
**3.1. Riesgos inherentes y defectos**  
- **Sesgos persistentes**  
  - Aunque procesen millones de textos, replican prejuicios presentes en los datos (por ejemplo, estereotipos de género o culturales).  
  - Si se usan para decisiones jurídicas o sociales, pueden reproducir o incluso amplificar injusticias previas.  
- **Falta de comprensión profunda**  
  - Los LLM “predicen” la próxima palabra según estadística, pero no “entienden” el significado en el sentido humano.  
  - Cuando se enfrentan a dilemas éticos complejos o a contexto histórico-sociocultural, su “razonamiento” carece de reflexividad genuina.  
- **Errores de factualidad (“hallucinations”)**  
  - Inventan citas, fechas o datos que suenan plausibles pero no corresponden a la realidad.  
  - Para decisiones urgentes (diagnósticos médicos, planes de contingencia), una respuesta inventada puede ser peligrosa.  

**3.2. Reflexiones filosóficas: ¿qué es “conocimiento” en un LLM?**  
- **Conocimiento estadístico vs. conocimiento proposicional**  
  - Un LLM acumula patrones lingüísticos, no verdades verificadas. Su “conocimiento” es una red de correlaciones, no un cuerpo organizado de proposiciones con fundamento en evidencia.  
- **Inteligencia aparente**  
  - Emula coherencia discursiva sin conciencia ni intencionalidad.  
  - La pregunta filosófica: ¿puede la imitación de respuestas correctas equipararse a la comprensión auténtica?  
- **Autoridad epistémica dudosa**  
  - Un experto humano puede argumentar sus razonamientos; un LLM no puede justificar con conciencia sus respuestas: carece de “razón de ser” más allá del entrenamiento estadístico.  

---

### 4. **Potencial futuro**
**4.1. Evolución tecnológica y metodológica**  
- **Hibridación humano–máquina**  
  - Modelos de “bucle de decisión” donde el LLM propone alternativas y un experto valida, corrige y contextualiza.  
  - Plataformas colaborativas en tiempo real: el LLM sugiere, el humano opina, el LLM refina.  
- **Mejor interpretación de contexto**  
  - Incorporación de “memoria de contexto extendida”: seguimiento de decisiones pasadas, historial de cambios y evaluación de impactos previos.  
  - Integración de datos estructurados (bases de datos, sensores IoT) junto al texto para enriquecer el contexto de decisión.  

**4.2. Avances en transparencia y explicabilidad**  
- **Modelos interpretables**  
  - Investigación en redes que expliquen por qué asocian ciertos conceptos; p. ej., “atención explicable” en Transformer.  
  - Paneles de auditoría interna que muestren cuáles documentos o frases influyeron en una recomendación.  
- **Curaduría de datos de entrenamiento**  
  - Bases de entrenamiento reguladas, auditables y clasificadas según criterios de calidad (no solo volumen).  
  - Políticas que limiten uso de fuentes no verificables o claramente sesgadas.  

**4.3. Nuevas aplicaciones en la toma de decisiones**  
- **Simulación de escenarios**  
  - Generación de análisis contrafactuales: “Si el parque plantara 500 árboles adicionales, ¿cómo cambiaría la captura de CO₂ en 5 años?”.  
  - Soporte en planificación urbana y diseño de políticas públicas, ensayando múltiples variables en paralelo.  
- **Monitoreo continuo y adaptación**  
  - Integración con sistemas de recolección de datos en tiempo real (cámaras, satélites, sensores ambientales) para que el LLM ajuste recomendaciones sobre la marcha.  
  - Algoritmos de retroalimentación: se valida en campo, se corrigen errores y el modelo aprende de datos recientes.

---

### 5. **Conclusión concreta**
- **Cuándo confiar**  
  - En tareas de **síntesis rápida de información**: revisión de literatura, resúmenes ejecutivos, primeras aproximaciones a un problema.  
  - En entornos donde la **consistencia estilística** y rapidez de respuesta sean prioritarias, siempre acompañadas de supervisión humana.  
- **Cuándo dudar**  
  - Para **decisiones críticas**: diagnósticos médicos definitivos, sentencias judiciales, políticas de alto impacto social.  
  - Cuando haya **alto riesgo de sesgo** o falta de actualización de datos, especialmente si el contexto local importa.  
- **Cómo balancear aporte y supervisión**  
  - Implementar un **sistema de validación tripartita**: LLM → experto humano → revisión externa (p. ej., comité independiente).  
  - Fomentar la **reflexión epistemológica continua**: comprender que, aunque un LLM pueda razonar “rápido”, no razona “con valores” ni asume responsabilidad ética.  
  - Reconocer el valor del LLM como **herramienta auxiliar**, no como autoridad autónoma. La supervisión humana no es un adorno: es la condición necesaria para que su uso sea legítimo.  

En última instancia, los LLM representan un avance significativo en la capacidad de procesar información y generar opciones, pero su verdadera utilidad en el mundo real depende de cómo se integren en redes de supervisión, ética y provisión de datos de alta calidad. La filosofía nos enseña a cuestionar la fuente del conocimiento y a no ceder el juicio crítico ante apariencias convincentes; así, un uso responsable de los LLM exige disciplina intelectual y compromiso con la verdad.